<a href="https://colab.research.google.com/github/eispoohw/CS493-Math-Methods-in-ML/blob/main/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

In [2]:
!gdown 1PRxLQ1YvXRylaTF8ks1a5BhGLFyoCA61
!gdown 1HDVBsj7p0AePBXs_S9qweDGU0s0htTVQ
!gdown 1cgGkHek94tauBpbtMN2CCQC0ACTOzQkv
!gdown 1olXMkeQuv4UV--b0bQFusqR-zf0keP8S

Downloading...
From: https://drive.google.com/uc?id=1PRxLQ1YvXRylaTF8ks1a5BhGLFyoCA61
To: /content/gender_train.csv
100% 99.9k/99.9k [00:00<00:00, 64.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1HDVBsj7p0AePBXs_S9qweDGU0s0htTVQ
To: /content/tr_mcc_codes.csv
100% 14.9k/14.9k [00:00<00:00, 15.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1cgGkHek94tauBpbtMN2CCQC0ACTOzQkv
To: /content/tr_types.csv
100% 14.2k/14.2k [00:00<00:00, 19.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1olXMkeQuv4UV--b0bQFusqR-zf0keP8S
To: /content/transactions.csv
100% 314M/314M [00:02<00:00, 107MB/s]


In [3]:
gender_train = pd.read_csv('gender_train.csv', sep=',')
tr_mcc_codes = pd.read_csv('tr_mcc_codes.csv', sep=';', index_col='mcc_code')
tr_types = pd.read_csv('tr_types.csv', sep=';', index_col='tr_type')
transactions = pd.read_csv('transactions.csv', sep=',', nrows=1000000)

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [4]:
full_sample = transactions.sample(1000)['tr_type']
full_sample.to_frame()

,tr_type
574612,2010
891377,7070
893982,1010
548193,1030
717989,7070
...,...
299488,1010
880382,7071
948879,1010
284400,2010


In [5]:
valid_sample = full_sample.loc[full_sample.isin(tr_types.index)]
satisfy = tr_types.loc[valid_sample]['tr_description'].str.contains('POS|АТМ', regex=True).sum()

res = satisfy / len(full_sample)

print(f"Sample length: {len(full_sample)}\n\
Valid indexes: {len(valid_sample)}\n\
Satisfying values: {satisfy}\n\
Result: {res}")

Sample length: 1000
Valid indexes: 1000
Satisfying values: 628
Result: 0.628


## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [6]:
freq = transactions['tr_type'].value_counts(normalize=True, ascending=False)
freq.to_frame()

,tr_type
1010,0.231117
2010,0.151166
7070,0.149006
1110,0.137658
1030,0.118975
...,...
2446,0.000004
4096,0.000004
8146,0.000003
1510,0.000002


In [7]:
largest_freq = freq.nlargest(10)
freq_desc = tr_types.loc[largest_freq.index]
freq_desc['tr_frequency'] = round(largest_freq, 3)
freq_desc

,tr_description,tr_frequency
1010,Покупка. POS ТУ СБ РФ,0.231
2010,Выдача наличных в АТМ Сбербанк России,0.151
7070,Перевод на карту (с карты) через Мобильный бан...,0.149
1110,Покупка. POS ТУ Россия,0.138
1030,Оплата услуги. Банкоматы СБ РФ,0.119
2370,Списание с карты на карту по операции <перевод...,0.050
7010,Взнос наличных через АТМ (в своем тер.банке),0.029
7030,Перевод на карту (с карты) через АТМ (в предел...,0.026
7071,Перевод на карту (с карты) через Мобильный бан...,0.016
1100,Покупка. ТУ Россия,0.015


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [8]:
# frame with both sums for each customer
customer_io = transactions[['customer_id', 'amount']].groupby(by='customer_id')['amount'].agg({
    ('income_sum', lambda x: sum([i for i in x if i > 0])),
    ('outcome_sum', lambda x: sum([-i for i in x if i < 0]))
})
customer_io

,income_sum,outcome_sum
customer_id,,
115617,786070.52,11764082.12
117556,2379547.78,11045837.36
127946,0.00,2265191.34
186730,17371260.17,19950843.61
189257,11695381.77,29380469.52
...,...,...
99864691,577200.37,572091.13
99900908,148230.44,6052983.42
99953525,1897798.84,6812421.89


In [9]:
# customer with the maximum income
princess = customer_io[['income_sum']].sort_values(by='income_sum', ascending=False).head(1)
princess

,income_sum
customer_id,
70780820,1.248115e+09


In [10]:
# customer with the maximum outcome
pauper = customer_io[['outcome_sum']].sort_values(by='outcome_sum', ascending=False).head(1)
pauper

,outcome_sum
customer_id,
70780820,1.249952e+09


In [11]:
# add column with difference between in- and outcomes
customer_io['difference'] = abs(customer_io.outcome_sum - customer_io.income_sum)

pauper_princess_idxes = [pauper.index, princess.index]
customer_io[customer_io.index.isin(pauper_princess_idxes)]

,income_sum,outcome_sum,difference
customer_id,,,
70780820,1.248115e+09,1.249952e+09,1.837318e+06


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [12]:
agg_func = {'amount': ['mean', 'median']}

In [13]:
# tr_type's mean and median 
appropriate = transactions[transactions.tr_type.isin(largest_freq.index)][['tr_type', 'amount']]
appropriate.groupby("tr_type").agg(agg_func).round(2)

amount           
              mean     median
tr_type                      
1010     -19784.75   -7411.52
1030      -5320.98   -2245.92
1100     -44061.83  -10188.26
1110     -32119.33  -11207.57
2010    -136077.63  -44918.32
2370    -205418.25  -44918.32
7010     276391.79  112295.79
7030      86104.33   13951.52
7070      65569.83   11319.42
7071      66806.83    3593.47

In [14]:
# customer's tr_type mean and median
appropriate = transactions[transactions.customer_id.isin(pauper_princess_idxes)][['customer_id', 'tr_type', 'amount']]
appropriate.groupby(['customer_id', 'tr_type']).agg(agg_func).round(2)

amount            
                           mean      median
customer_id tr_type                        
70780820    2010    -2941055.87 -3368873.66
            2330    -2382398.34 -2245915.77
            2370    -2218504.99 -2245915.77
            6110       17562.93    10028.01
            7020       14651.29    10319.98
            7021       28500.67    28500.67
            7030       14061.96     8848.91
            7031       19210.36    12992.62
            7034       14306.48    14306.48
            7040       21538.33    14396.32
            7070       13848.21     8714.15
            7071       18600.57    12543.44
            7074       22811.59     9387.93

## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [15]:
nt = transactions.merge(gender_train, how='left', on='customer_id')
nt = nt.merge(tr_types, how='inner', on='tr_type')
nt = nt.merge(tr_mcc_codes, how='inner', on='mcc_code')
nt.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [16]:
agg_func_amount = {
    ('income', lambda x: np.mean([i for i in x if i > 0])),
    ('outcome', lambda x: np.mean([-i for i in x if i < 0]))
}

# mean incomes and outcomes grouped by gender
in_out = nt.groupby(['gender'])['amount'].agg(agg_func_amount).round(2)

abs(in_out.diff().reset_index(drop=True)).iloc[1].rename('difference').to_frame()

,difference
outcome,32718.05
income,63366.58


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [17]:
incomes = nt[nt.amount > 0].groupby(['gender', 'tr_type']).max('amount').rename(columns={'amount': 'max_income'})
smallest = incomes.groupby(level=0, group_keys=False).apply(lambda x: x.nsmallest(10, ['max_income']))
smallest

customer_id  mcc_code  max_income
gender tr_type                                   
0.0    4051        65368200      4829     1122.96
       4110        14808847      6011     2245.92
       4210        67193294      6011     2245.92
       2370        44552122      4829     2335.75
       4100        34267056      6011     5041.86
       7075        14169381      6010     6737.75
       2110        14808847      6011    22459.16
       1010        69189450      7832    24839.83
       2010        69189450      6011    44918.32
       2210        67193294      6011    68315.82
1.0    2020        46579237      6010     2245.92
       4110        99048480      6011     2245.92
       1010        40877619      5411    28803.87
       8100        47653609      5813    64682.37
       6000        61024099      3000    79280.83
       7041         6839194      6010    87590.72
       2010        91582559      6011    89836.63
       2110        99048480      6011   112295.79
       2370        56053006      4829   113194.15
       7015        66245004      6011   121279.45

In [18]:
smallest[smallest.index.get_level_values(1).duplicated()].reset_index()[['tr_type']]

,tr_type
0,4110
1,1010
2,2010
3,2110
4,2370


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [19]:
outcomes = nt[nt.amount < 0][['gender', 'mcc_code', 'amount']]
outcomes = outcomes.groupby(['gender', 'mcc_code'], as_index=True).sum(['amount'])
outcomes

amount
gender mcc_code             
0.0    742        -386207.68
       1711       -674897.69
       1799       -261626.73
       2741        -20746.42
       3000     -74431645.27
...                      ...
1.0    8999     -28643346.63
       9222       -988205.17
       9311       -373271.22
       9399      -1810820.67
       9402         -4581.67

[349 rows x 1 columns]

In [20]:
outcomes_fem = outcomes[outcomes.index.get_level_values(0) == 0].reset_index(level=0, drop=True)[['amount']]
outcomes_male = outcomes[outcomes.index.get_level_values(0) == 1].reset_index(level=0, drop=True)[['amount']]

res = outcomes_fem.merge(outcomes_male, how='inner', on='mcc_code')
res['difference'] = abs(res.amount_x - res.amount_y)
res[['difference']].nlargest(10, 'difference').merge(tr_mcc_codes, how='inner', on='mcc_code')

,difference,mcc_description
mcc_code,,
6011,3.085862e+09,Финансовые институты — снятие наличности автом...
4829,2.499153e+09,Денежные переводы
6010,2.208580e+08,Финансовые институты — снятие наличности вручную
5541,1.554326e+08,Станции техобслуживания
5511,7.352877e+07,"Легковой и грузовой транспорт — продажа, серви..."
5812,7.037856e+07,"Места общественного питания, рестораны"
5977,4.241762e+07,Магазины косметики
5533,4.233808e+07,Автозапчасти и аксессуары
7995,4.079564e+07,Транзакции по азартным играм


## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 

In [21]:
with_hour = nt
with_hour['tr_hour'] = [int(x[-1][:2]) for x in with_hour.tr_datetime.str.split(' ')]
with_hour.sample(10)

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,tr_description,mcc_description,tr_hour
850510,70780820,405 14:00:25,6010,7070,2223.46,888993,1.0,Перевод на карту (с карты) через Мобильный бан...,Финансовые институты — снятие наличности вручную,14
806232,70780820,169 12:57:08,6010,7070,11184.66,NaN,1.0,Перевод на карту (с карты) через Мобильный бан...,Финансовые институты — снятие наличности вручную,12
307781,14114778,292 12:14:58,6011,2011,-13475.49,110479,1.0,Выдача наличных в АТМ Сбербанка (в других ТБ),Финансовые институты — снятие наличности автом...,12
985356,30388707,294 00:00:00,4112,1110,-5412.66,04140252,0.0,Покупка. POS ТУ Россия,Пассажирские железные перевозки,0
711529,81677726,181 13:02:56,5661,1110,-45313.60,NaN,0.0,Покупка. POS ТУ Россия,Обувные магазины,13
971404,43442929,299 00:00:00,7922,1100,-59292.18,24031101,0.0,Покупка. ТУ Россия,Театральные продюсерские агентства,0
37294,42568320,281 19:01:40,4814,1030,-2245.92,889001,0.0,Оплата услуги. Банкоматы СБ РФ,"Звонки с использованием телефонов, считывающих...",19
930745,70780820,341 11:41:42,6010,7030,6558.07,613715,1.0,Перевод на карту (с карты) через АТМ (в предел...,Финансовые институты — снятие наличности вручную,11
310258,2208164,2 13:35:44,6011,4011,-673.77,NaN,0.0,Плата за получение наличных в АТМ Сбербанка (в...,Финансовые институты — снятие наличности автом...,13
584906,27868636,265 17:03:02,5411,1010,-1515.99,873174,1.0,Покупка. POS ТУ СБ РФ,"Бакалейные магазины, супермаркеты",17


In [22]:
with_hour[(with_hour.amount < 0) & (with_hour.tr_hour < 6)].groupby('gender').agg({'amount': 'count'})

,amount
gender,
0.0,38827
1.0,43017


# Дополнительное задание
По каждой категории найдите сумму трат в ночное время и в остальное время. Найдите соотношение одного к другому. Выведите список 10 категорий с описаниями, упорядоченных по убываний отношения ночных трат к остальным тратам.  

In [49]:
hour_spends = with_hour[with_hour.amount < 0][['tr_type', 'tr_hour', 'amount']]

night_spends = hour_spends[hour_spends.tr_hour < 6].groupby(['tr_type'])['amount'].agg({('night', 'sum')})
day_spends = hour_spends[hour_spends.tr_hour >= 6].groupby(['tr_type'])['amount'].agg({('day', 'sum')})

hour_spends = night_spends.merge(day_spends, how='outer', on='tr_type')#.fillna(1)
hour_spends

,night,day
tr_type,,
1010,-1.430785e+08,-4.429580e+09
1030,-5.285342e+07,-5.802102e+08
1100,-5.505117e+08,-1.369127e+08
1110,-3.023499e+09,-1.409416e+09
1200,-3.550967e+08,-1.722282e+07
1210,-4.113457e+08,-1.023736e+08
1410,-3.548820e+06,-6.670475e+05
1510,-2.239605e+04,NaN
2010,-1.141155e+09,-1.943015e+10


In [54]:
hour_spends['frac'] = abs(hour_spends['night'] / hour_spends['day'])
hour_spends[['frac']].nlargest(10, 'frac').merge(tr_types, how='left', on='tr_type')

,frac,tr_description
tr_type,,
2200,280.487670,Наличные. Зарубеж. банк
1200,20.617806,Покупка. Зарубеж. ТУ
2100,14.341184,Наличные. Россия
4200,12.986013,Плата за получение наличных. Зарубеж.
4100,7.694491,Плата за получение наличных. Россия
1410,5.320191,н/д
1100,4.020896,Покупка. ТУ Россия
1210,4.018083,Покупка. POS Зарубеж. ТУ
4110,2.792048,Плата за получение наличных в АТМ. Россия
